In [286]:
! pip install spacy
! python -m spacy download ru_core_news_sm
! python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     --------------------------------------- 0.1/15.3 MB 550.5 kB/s eta 0:00:28
     --------------------------------------- 0.1/15.3 MB 819.2 kB/s eta 0:00:19
     --------------------------------------- 0.2/15.3 MB 985.7 kB/s eta 0:00:16
      --------------------------------------- 0.3/15.3 MB 1.1 MB/s eta 0:00:14
      --------------------------------------- 0.4/15.3 MB 1.3 MB/s eta 0:00:12
     - -------------------------------------- 0.4/15.3 MB 1.2 MB/s eta 0:00:13
     - -------------------------------------- 0.6/15.3 MB 1.5 MB/s eta 0:00:10
     - -------------------------------------- 0.7/15.3 MB 1.7 MB/s eta 0:00:09
     -- ------------------------------------- 1.0/15.3 MB 2.2 MB/s eta 0:00:07
     --- ------------------------------------ 1.2/15.3 MB 2.4 MB/s eta 0:00:06
     --- ------------------------------------ 1.2/15.3 MB 2.4 M

In [287]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


# Импорты библиотек

In [288]:
import os

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from collections import Counter
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from string import punctuation
import spacy
from langdetect import detect

import openai

In [289]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [290]:
openai.api_key = 'API-KEY here'

# Подготовка к работе

Загружаем модели и создаём нужные списки

In [ ]:
nlp_eng = spacy.load('en_core_web_sm', disable=['ner', 'parser'])
nlp_rus = spacy.load('ru_core_news_sm', disable=['ner', 'parser'])

stop_words_rus = stopwords.words('russian')
stop_words_eng = stopwords.words('english')
stop_words = stop_words_rus+stop_words_eng+['каждый день', 'каждый', 'день', 'красная цена', 'красная', 'цена', 'верный', 'дикси', 'моя', 'моя цена', 'окей','то, что надо!', 'smart','spar', 'ашан']

Создаём категории товаров

In [291]:
items = ['автозапчасти', 'видеоигры', 'напитки', 'продукты питания', 'закуски и приправы', 'аквариум', 'одежда', 'уборка', 'образование', 'электроника']

Функция для создания очищенных предложений и колонки с ними

In [292]:
def filter_text(data):

    filtered_data = data.copy()

    selected_posts = []
    all_sentences = []

    for index in range(len(filtered_data["sale"])):
        value = filtered_data.loc[index, "sale"]
        try:
            lang = detect(value)

            if lang == 'en':
                doc = nlp_eng(value)
                lemmas = [token.lemma_ for token in doc if token.is_alpha and token.text not in punctuation]
                cleaned_sentence = ' '.join(lemmas)
                filtered_data.loc[index, 'cleaned_sale'] = cleaned_sentence
            else:
                doc = nlp_rus(value)
                lemmas = [token.lemma_ for token in doc if token.is_alpha and token.text not in punctuation]
                cleaned_sentence = ' '.join(lemmas)
                filtered_data.loc[index, 'cleaned_sale'] = cleaned_sentence
        except Exception as e:
            print(f"Error processing value: {value}")
            print(f"Error message: {str(e)}")

    filtered_data = filtered_data.dropna(subset=['cleaned_sale'])

    return filtered_data

In [ ]:
Функция для создания топиков

In [313]:
def topics(data):
    sentences = data['cleaned_sale']
    system = f'{items} из перечисленных категорий, ТОЧНО выбери одну и укажи ТОЛЬКО её (без знаков препинания и дополнительных слов). Новых предлагать нельзя Регистр должен быть тем же. Только из списка'

    if len(sentences) > 50000:
        n_topics = 3
    elif len(sentences) > 25000 and len(sentences) <= 50000:
        n_topics = 2
    elif len(sentences) > 300 and len(sentences) <= 25000:  
        vectorizer = CountVectorizer(stop_words=stop_words, max_features=15, ngram_range=(5,5))
        x = vectorizer.fit_transform(sentences)
        tokens = vectorizer.get_feature_names_out()
        print("Топ популярных токенов:")
        print(", ".join(tokens))
        content = f'{items} выбери категорию на основе списка слов ниже: {", ".join(tokens)}'
        completion = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[{"role": "system", "content": system},
            {"role": "user", "content": content}], max_tokens= 20, temperature=0.05)
        response_content = completion["choices"][0]["message"]["content"]
        data['topic'] = response_content
        return data
    else:
        data['topic'] = 'нет категории'
        return data

    tfidf = TfidfVectorizer(stop_words=stop_words, min_df=20, max_df=0.95, ngram_range=(3,3))
    x = tfidf.fit_transform(sentences)
    id2word = {i: token for i, token in enumerate(tfidf.get_feature_names_out())}
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=12345)
    topics = lda.fit_transform(x)
    
    topic_names=[]
    for i in range(n_topics):
        main_topics = pd.DataFrame()
        c = lda.components_[i, :]
        topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-10:]]
        print("Тема", i+1, ":")
        main_topics.loc[i, 'Тема'] = i+1
        print(", ".join(topic_tokens))
        main_topics.loc[i, 'Ключевые слова'] = ", ".join(topic_tokens)
        content = f'{items} выбери категорию на основе списка слов ниже: {", ".join(topic_tokens)}'
        completion = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[{"role": "system", "content": system},
            {"role": "user", "content": content}], max_tokens= 20, temperature=0.05)
        response_content = completion["choices"][0]["message"]["content"]
        topic_names.append(response_content)
        print("\n")

    topic_assignments = lda.transform(x)
    data['topic'] = [topic_names[i] for i in np.argmax(topic_assignments[:len(data)], axis=1) % n_topics]
        
    return data

# Предобработка данных

In [294]:
pth1 = 'final_ds_finnopolis.csv'
pth2 = '/datasets/final_ds_finnopolis.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
else:
    print('Something is wrong')

In [295]:
display(data.head(20))

,"Оперативная память apacer nox [ah4u16g32c28ymbaa-1] 16 гб [ddr4, 16 гбx1 шт, 3200 мгц, 16-20-20-38]",1838685550,4399,Блахов Василий Александрович
0,Видеокарта kfa2 geforce rtx 3050 x black [35ns...,1227447308,29199,Блахов Василий Александрович
1,Оперативная память kingston fury beast black r...,1838685550,3599,Блахов Василий Александрович
2,Свеча зажигания ngk 4549 cr7hsa 1 шт.,1085769503,1092,Блахов Василий Александрович
3,Газовые упоры капота geely coolray / джили кул...,1197983357,678,Блахов Василий Александрович
4,Фильтр салонный marshall mc4410 для chevrolet ...,1197983357,453,Блахов Василий Александрович
5,Бачок general motors 13118164 черный 1 шт.,1197983357,3702,Блахов Василий Александрович
6,Тормозной диск передний marshall m2000405 для ...,1197983357,568,Блахов Василий Александрович
7,Натяжитель цепи газ (дв. змз. 406) гидравличес...,1197983357,912,Блахов Василий Александрович
8,Таз овальный hausmann funny smile 12 л в ассор...,1505938033,179,Блахов Василий Александрович
9,Средство чистящее cillit bang антипятна и гиги...,1505938033,289,Блахов Василий Александрович


In [296]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299999 entries, 0 to 299998
Data columns (total 4 columns):
 #   Column                                                                                               Non-Null Count   Dtype 
---  ------                                                                                               --------------   ----- 
 0   Оперативная память apacer nox [ah4u16g32c28ymbaa-1] 16 гб [ddr4, 16 гбx1 шт, 3200 мгц, 16-20-20-38]  299999 non-null  object
 1   1838685550                                                                                           299999 non-null  int64 
 2   4399                                                                                                 299999 non-null  int64 
 3   Блахов Василий Александрович                                                                         299999 non-null  object
dtypes: int64(2), object(2)
memory usage: 9.2+ MB


In [297]:
data.describe()

,1838685550,4399
count,2.999990e+05,299999.000000
mean,1.593975e+09,2439.293104
std,2.934788e+08,12441.891755
min,1.085770e+09,0.000000
25%,1.363498e+09,102.000000
50%,1.505938e+09,231.000000
75%,1.964649e+09,640.000000
max,1.983695e+09,258999.000000


Посмотрим на уникальные значения

In [298]:
fio = data['Блахов Василий Александрович'].unique()

In [299]:
len(fio)

9972

In [300]:
categories = data['1838685550'].unique()

In [301]:
len(categories)

23

In [302]:
sales = data['Оперативная память apacer nox [ah4u16g32c28ymbaa-1] 16 гб [ddr4, 16 гбx1 шт, 3200 мгц, 16-20-20-38]'].unique()

In [303]:
len(sales)

7232

Посмотрим, не является ли ноль в цене ошибкой

In [304]:
data[data['4399']==0].head(15)

,"Оперативная память apacer nox [ah4u16g32c28ymbaa-1] 16 гб [ddr4, 16 гбx1 шт, 3200 мгц, 16-20-20-38]",1838685550,4399,Блахов Василий Александрович
1083,Как создаются игры. основы разработки для начи...,1310763416,0,Городнина Татьяна Викторовна
1745,Кокетливый интеллект. как научить искусственны...,1310763416,0,Долгов Юрий Сергеевич
5316,Архитектура видеоигровых миров. уровень пройден!,1310763416,0,Арюлина Ираида Анатольевна
11917,Php и mysql. серверная веб-разработка,1310763416,0,Базяев Андрей Васильевич
17226,Как создаются игры. основы разработки для начи...,1310763416,0,Пыхунова Инна Николаевна
17614,Кокетливый интеллект. как научить искусственны...,1310763416,0,Калугин Олег Михайлович
19111,Призвание - геймер. как построить карьеру в ин...,1310763416,0,Кажурин Виталий Викторович
21126,"Не лги себе. почему big data знает тебя лучше,...",1310763416,0,Берглезова Людмила Федоровна
21330,Призвание - геймер. как построить карьеру в ин...,1310763416,0,Соснина Алена Валерьевна
23643,Php и mysql. серверная веб-разработка,1310763416,0,Кургузов Илья Александрович


Нет, это бесплатные курсы

Создаём нормальные колонки

In [305]:
columns = ['sale', 'category', 'price', 'client']

In [306]:
data = pd.concat([pd.DataFrame([data.columns], columns=data.columns), data]).reset_index(drop=True)
data.columns=columns

In [307]:
display(data.head())

,sale,category,price,client
0,Оперативная память apacer nox [ah4u16g32c28ymb...,1838685550,4399,Блахов Василий Александрович
1,Видеокарта kfa2 geforce rtx 3050 x black [35ns...,1227447308,29199,Блахов Василий Александрович
2,Оперативная память kingston fury beast black r...,1838685550,3599,Блахов Василий Александрович
3,Свеча зажигания ngk 4549 cr7hsa 1 шт.,1085769503,1092,Блахов Василий Александрович
4,Газовые упоры капота geely coolray / джили кул...,1197983357,678,Блахов Василий Александрович


## Топик моделинг

Применяем первую фильтрующую функцию

In [309]:
df_filtred = filter_text(data)

In [310]:
df_filtred.to_csv('df_filtred.csv')

In [311]:
df_filtred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   sale          300000 non-null  object
 1   category      300000 non-null  object
 2   price         300000 non-null  object
 3   client        300000 non-null  object
 4   cleaned_sale  300000 non-null  object
dtypes: object(5)
memory usage: 11.4+ MB


Всё прошло хорошо. Теперь применим вторую функцию.

In [314]:
processed_dfs = [] 
for category, group_df in df_filtred.groupby('category'): 
    print(len(group_df)) 
    processed_group_df = topics(group_df) 
    processed_dfs.append(processed_group_df) 
    
final_df = pd.concat(processed_dfs, ignore_index=True)

8734
Топ популярных токенов:
autoprofi bat площадь сечение жила, bat площадь сечение жила медь, hans pries комплект стойка стабилизатор, pries комплект стойка стабилизатор замена, акб autoprofi bat площадь сечение, двойн изоляция крепление клемма болт, жила медь двойн изоляция крепление, изоляция крепление клемма болт доп, клемма болт доп вывод см, крепление клемма болт доп вывод, медь двойн изоляция крепление клемма, площадь сечение жила медь двойн, провод акб autoprofi bat площадь, сечение жила медь двойн изоляция, соединительный провод акб autoprofi bat
9104
Топ популярных токенов:
bm mazda gj mazda ke, dacia nissan renault lada largus, gj mazda ke kf кросс, marshall mazda bm mazda gj, mazda bm mazda gj mazda, mazda gj mazda ke kf, mazda ke kf кросс номер, дверь резиновый комплект штука ваз, лампа галогенный маяк ultra super, огонь drl avs светодиод шт, передний dacia nissan renault lada, подшипник ступица передний dacia nissan, проём дверь резиновый комплект штука, ступица передний

Видим в целом хорошие и правильные топики. Посмотрим как их определила нейросеть.

In [315]:
display(final_df.head())

,sale,category,price,client,cleaned_sale,topic
0,Свеча зажигания ngk 4549 cr7hsa 1 шт.,1085769503,1092,Блахов Василий Александрович,свеча зажигание ngk шт,автозапчасти
1,Зарядное устройство avs bt-6030 для автомобиль...,1085769503,741,Борзенко Татьяна Александровна,зарядный устройство avs для автомобильный акку...,автозапчасти
2,Свеча зажигания ngk 3981 br9eix 1 шт.,1085769503,996,Мешкова Валерия Николаевна,свеча зажигание ngk шт,автозапчасти
3,Свеча зажигания ngk 2467 ptr5a-13 1 шт.,1085769503,660,Мешкова Валерия Николаевна,свеча зажигание ngk шт,автозапчасти
4,Lada колодки тормозные задние lada 21920350209...,1085769503,5444,Мешкова Валерия Николаевна,lada колодка тормозной задний lada комплект,автозапчасти


In [316]:
final_df['topic'].unique()

array(['автозапчасти', 'аквариум', 'электроника', 'видеоигры',
       'продукты питания', 'уборка', 'одежда', 'закуски и приправы',
       'напитки', 'нет категории'], dtype=object)

Лишних категорий и дублей нет. Отлично!

In [317]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   sale          300000 non-null  object
 1   category      300000 non-null  object
 2   price         300000 non-null  object
 3   client        300000 non-null  object
 4   cleaned_sale  300000 non-null  object
 5   topic         300000 non-null  object
dtypes: object(6)
memory usage: 13.7+ MB


# Группировка данных и сохранение финальных датасетов

Создадим датасет, где сгруппируем покупки клиентов по категориям. Посмотрим на частотность и общую стоимость покупок в категориях

In [318]:
cashbacks = final_df.groupby(['client', 'topic']).agg({'topic': 'count', 'price': 'sum'})

In [319]:
cashbacks.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 64259 entries, ('Абакелия Мукас Робертовна', 'видеоигры') to ('Яшкова Надежда Алексеевна', 'электроника')
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   topic   64259 non-null  int64 
 1   price   64259 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [320]:
cashbacks = cashbacks.rename(columns={'topic': 'frequency', 'price': 'total_price'})

In [321]:
display(cashbacks.sort_values(['client', 'frequency'], ascending=[True, False]))

frequency total_price
client                    topic                                    
Абакелия Мукас Робертовна видеоигры                   6        2395
                          напитки                     6         844
                          продукты питания            6        1285
                          уборка                      6        1894
                          закуски и приправы          3         222
...                                                 ...         ...
Яшкова Надежда Алексеевна напитки                     5         868
                          уборка                      5        1389
                          аквариум                    3        8153
                          закуски и приправы          3         752
                          одежда                      1         599

[64259 rows x 2 columns]

In [322]:
final_df.to_csv('final_df.csv') 
final_df.to_excel('final_df.xlsx')

In [324]:
cashbacks.to_csv('cashbacks.csv') 
cashbacks.to_excel('cashbacks.xlsx')